# Tarea 5

### Ejercicio 1

In [5]:
import pulp as pp

Definiendo el modelo

In [6]:
#Modelo
modelo= pp.LpProblem(name="supplement-problem", sense=pp.LpMinimize)
#Variables
VegaVita = pp.LpVariable(name="VegaVita", lowBound=0, cat='Integer')
HappyHealth = pp.LpVariable(name="HappyHealth", lowBound=0, cat='Integer')
#Pendientes
cost_VegaVita = 0.20
cost_HappyHealth = 0.30
obj_func = cost_VegaVita * VegaVita + cost_HappyHealth * HappyHealth
#Parámetros
vitC_req = 60
calcium_req = 1000
iron_req = 18
niacin_req = 20
magnesium_req = 360

vitC_VegaVita = 20
calcium_VegaVita = 500
iron_VegaVita = 9
niacin_VegaVita = 2
magnesium_VegaVita = 60

vitC_HappyHealth = 30
calcium_HappyHealth = 250
iron_HappyHealth = 2
niacin_HappyHealth = 10
magnesium_HappyHealth = 90

In [7]:
# Requerimientos Vitamina
C1 = pp.LpConstraint(name='Vitamin C Constraint',
                     e=vitC_VegaVita*VegaVita + vitC_HappyHealth*HappyHealth,
                     rhs=vitC_req,
                     sense=pp.LpConstraintGE)

# Requerimientos de calcio
C2 = pp.LpConstraint(name='Calcium Constraint',
                     e=calcium_VegaVita*VegaVita + calcium_HappyHealth*HappyHealth,
                     rhs=calcium_req,
                     sense=pp.LpConstraintGE)

# Requerimientos de hierro
C3 = pp.LpConstraint(name='Iron Constraint',
                     e=iron_VegaVita*VegaVita + iron_HappyHealth*HappyHealth,
                     rhs=iron_req,
                     sense=pp.LpConstraintGE)

# Requerimientos de niacinamida
C4 = pp.LpConstraint(name='Niacin Constraint',
                     e=niacin_VegaVita*VegaVita + niacin_HappyHealth*HappyHealth,
                     rhs=niacin_req,
                     sense=pp.LpConstraintGE)

# Requerimientos de magnesio
C5 = pp.LpConstraint(name='Magnesium Constraint',
                     e=magnesium_VegaVita*VegaVita + magnesium_HappyHealth*HappyHealth,
                     rhs=magnesium_req,
                     sense=pp.LpConstraintGE)

In [8]:
#Agregando los parámetros
model+=obj_func
model += C1
model += C2
model += C3
model += C4
model += C5

In [13]:
solver_list = pp.listSolvers()
solver_list

['GLPK_CMD',
 'PYGLPK',
 'CPLEX_CMD',
 'CPLEX_PY',
 'GUROBI',
 'GUROBI_CMD',
 'MOSEK',
 'XPRESS',
 'XPRESS',
 'XPRESS_PY',
 'PULP_CBC_CMD',
 'COIN_CMD',
 'COINMP_DLL',
 'CHOCO_CMD',
 'MIPCL_CMD',
 'SCIP_CMD',
 'FSCIP_CMD',
 'SCIP_PY',
 'HiGHS',
 'HiGHS_CMD',
 'COPT',
 'COPT_DLL',
 'COPT_CMD']

In [23]:
#Solucion 
model.solve()

1

In [17]:
#Los resultados
Results = {"Model Status": pp.LpStatus[model.status]}
Results.update({"Optimal Solution": pp.value(model.objective)})
Results.update({v.name: v.varValue for v in model.variables()})
print(Results)

{'Model Status': 'Optimal', 'Optimal Solution': 1.2000000000000002, 'HappyHealth': 2.0, 'VegaVita': 3.0}


### Ejercicio 2

In [24]:
from ahp.ahp import AHP

In [25]:
import pandas as pd
pairwise_costliving=pd.read_excel("ahp_masters.xlsx",sheet_name='costliving', index_col=0)
pairwise_academics=pd.read_excel("ahp_masters.xlsx",sheet_name='academics', index_col=0)
pairwise_jobopport=pd.read_excel("ahp_masters.xlsx",sheet_name='jobopport', index_col=0)
pairwise_language=pd.read_excel("ahp_masters.xlsx",sheet_name='language', index_col=0)
pairwise_criteria=pd.read_excel("ahp_masters.xlsx",sheet_name='criteria', index_col=0)

In [26]:
pairwise_criteria

,costliving,academics,jobopport,language
costliving,1.000000,2.000000,4.000000,6
academics,0.500000,1.000000,3.000000,4
jobopport,0.250000,0.333333,1.000000,3
language,0.166667,0.250000,0.333333,1


In [27]:
#Siguiendo el mismo orden 
data = [pairwise_costliving.values.tolist(),
        pairwise_academics.values.tolist(),
        pairwise_jobopport.values.tolist(),
        pairwise_language.values.tolist(),
        pairwise_criteria.values.tolist()]

Implementando el modelo

In [28]:
ahp = AHP()
ahp.fit(data)

Obteniendo la mejor solución

In [29]:
Solution=pd.DataFrame(index=pairwise_jobopport.index)
Solution['Score']=ahp.classificate()
Solution['Ranking']=Solution.Score.rank(ascending=False)
Solution

,Score,Ranking
Brazil,0.330573,1.0
USA,0.242286,2.0
France,0.138482,5.0
England,0.145873,3.0
Germany,0.142786,4.0
